# Calculate probabilistic impact yearset

This module generates an `annual_impacts` object which contains probabilistic annual impacts for a specified amount of years (`sampled_years`). The impact values are extracted from a given `event_impact` object that contains impact values per event. The amount of `sampled_years` can be specified as an integer or as a list of years to be sampled for. The amount of events per sampled year (`events_per_year`) are determined with a Poisson distribution centered around n_events per year (n_events = sum(event_impacts.frequency). Then, the probabilistic events occuring in each sampled year (`selected_event`) are sampled uniformaly from the input `event_impacs` object and summed up per year (`impacts_per_year`). Thus, the `annual_impacts` object contains the sum of sampled (event) impacts for each sampled year. In contrast to the expected annual impact (eai), an `annual_impacts` object contains an impact for EACH sampled year and this value differs among years. The selected_events and the number of events_per_year are saved in a sampling dictionary (`sampling_dict`). 

The function impact_yearsets performs all these computational steps, taking an event_impacts object and the number of sampled_years as input. The output of the function is the annual_impacts object and the sampling_dict.
Moreover, a sampling_dict (generated in a previous run) can be provided as optional input and the user can decide whether a correction factor shall be applied (the default is applying the correction factor).
Reapplying the same sampling_dict does not only allow to reproduce the generated annual_impacts object, but also for a physically consistent way of sampling impacts caused by different hazards. 
The correction factor that is applied when the optional input `correction_fac`= True is a scaling of the computed annual_impacts that assures that the eai(annual_events) = eai(events_impacts).

To make the process more transparent, this tutorial shows the single computations that are performed when generating an annual_impacts object for a dummy event_impacts object. 

In [8]:
import numpy as np

import climada.util.yearsets as yearsets
from climada.engine import Impact

# dummy event_impacts object containing 10 event_impacts with the values 10-110 
# and the frequency 0.2 (Return period of 5 years)
event_impacts = Impact()
event_impacts.at_event = np.arange(10,110,10)
event_impacts.frequency = np.array(np.ones(10)*0.2)

# the number of years to sample impacts for (length(annual_impacts.at_event) = sampled_years)
sampled_years = 10

n_annual_events = np.sum(event_impacts.frequency)
n_input_events = len(event_impacts.at_event)
sampling_dict = yearsets.create_sampling_dict(sampled_years, n_annual_events, n_input_events)

impact_per_year = yearsets.compute_annual_impacts(event_impacts, sampling_dict)

correction_factor = yearsets.calculate_correction_fac(impact_per_year, event_impacts)

# compare the resulting annual_impacts with our step-by-step computation without applying the correction factor: 
    
annual_impacts, sampling_vect = yearsets.impact_yearset(event_impacts,
                                                        sampling_dict=sampling_dict,
                                                        correction_fac = False)

print('The annual_impacts.at_event values equal our step-by-step computed impacts_per year:')
print('annual_impacts.at_event = ', annual_impacts.at_event)
print('impact_per_year = ', impact_per_year)

# and here the same comparison with applying the correction factor (default settings):
annual_impacts, sampling_vect = yearsets.impact_yearset(event_impacts,
                                                        sampling_dict=sampling_dict)

print('The same can be shown for the case of applying the correction factor.' 
      'The annual_impacts.at_event values equal our step-by-step computed impacts_per year:')
print('annual_impacts.at_event = ', annual_impacts.at_event)
print('impact_per_year = ', impact_per_year/correction_factor)




2021-04-01 11:47:04,148 - climada.util.yearsets - INFO - The correction factor amounts to -25.170068027210878
The annual_impacts.at_event values equal our step-by-step computed impacts_per year:
annual_impacts.at_event =  [390.  90. 300. 190. 100.   0.   0. 240.  60. 100.]
impact_per_year =  [390.  90. 300. 190. 100.   0.   0. 240.  60. 100.]
2021-04-01 11:47:04,154 - climada.util.yearsets - INFO - The correction factor amounts to -25.170068027210878
The same can be used for the case of applying the correction factor.The annual_impacts.at_event values equal our step-by-step computed ipacts_per year:
annual_impacts.at_event =  [521.18181818 120.27272727 400.90909091 253.90909091 133.63636364
   0.           0.         320.72727273  80.18181818 133.63636364]
impact_per_year =  [521.18181818 120.27272727 400.90909091 253.90909091 133.63636364
   0.           0.         320.72727273  80.18181818 133.63636364]
